# Comparison of MendelImpute against Minimac4 and Beagle5 on 1000 genome project

**Dynamic programming memory requirement:** 
+ Target data requies $people * snps * 4$ bytes of RAM
+ Reference haplotype data requires $haplotypes * snps * 4$ bytes of RAM
+ Redundant haplotype set for imputation target requires roughly
$people * windows * 1000$ (max haplotypes per win) $* 16 bytes$ of RAM

**Fast method memory requirement:** 
+ Target data requies $people * snps * 4$ bytes of RAM
+ Reference haplotype data requires $haplotypes * snps * 4$ bytes of RAM
+ Redundant haplotype set for imputation target requires roughly
$people * windows * snps$ bits of RAM


In [1]:
using Revise
using VCFTools
using MendelImpute
using GeneticVariation
using Random
using StatsBase

┌ Warning: no Manifest.toml file found, static paths used
└ @ Revise /Users/biona001/.julia/packages/Revise/439di/src/Revise.jl:1108


# Filter data into target and reference data

+ `ref_chr22.vcf.gz`: haplotype reference files
+ `tgt_chr22.vcf.gz`: complete genotype information
+ `tgt_masked_chr22.vcf.gz`: the same as `tgt_chr22.vcf.gz` except some entries are masked
+ `tgt_masked_unphased_chr22.vcf.gz`: the same as `tgt_chr22.vcf.gz` except some entries are masked and heterozygote genotypes (1|0 or 0|1) are all changed to 1/0. 

In [10]:
"""
    filter_and_mask(data::String, samples::Int)

Creates reference haplotypes and (unphased) target genotype files from `data`. Duplicate
records (SNPs) are filtered out as well. 

# Inputs
`data`: The full (phased) data
`samples`: Number of samples (genotypes) desired in target file. Remaining haplotypes will become the reference panel
"""
function filter_and_mask(data::String, samples::Int; missingprop::Float64 = 0.01)
    n = nsamples(data)
    samples > n && error("requested samples exceed total number of genotypes in $data.")

    # output filenames (tgt_chr22.vcf.gz, ref_chr22.vcf.gz, and tgt_masked_chr22.vcf.gz)
    file = split(data, '.')[4] * ".vcf.gz"
    tgt = "./tgt_" * file
    ref = "./ref_" * file
    tgt_mask = "./tgt_masked_" * file
    tgt_mask_unphase = "./tgt_masked_unphased_" * file

    # compute target and reference index
    println("computing tgt/ref/record index")
    tgt_index = falses(n)
    tgt_index[sample(1:n, samples, replace=false)] .= true
    ref_index = .!tgt_index
    record_index = .!find_duplicate(data) # save only unique records (SNPs) 

    # generate masking matrix with `missingprop`% of trues (true = convert to missing)
    p = sum(record_index)
    masks = falses(p, samples)
    for j in 1:samples, i in 1:p
        rand() < missingprop && (masks[i, j] = true)
    end

    # create outputs 
    println("generating target file")
    VCFTools.filter(data, record_index, tgt_index, des = tgt)
    println("generating reference file")
    VCFTools.filter(data, record_index, ref_index, des = ref)
    println("generating masked file")
    mask_gt(tgt, masks, des=tgt_mask)

    # finally, unphase the target data
    unphase(tgt_mask, outfile=tgt_mask_unphase)
end

filter_and_mask

In [11]:
Random.seed!(2020)
cd("/Users/biona001/.julia/dev/MendelImpute/data/1000_genome_phase3_v5/filtered")

chr22 = "../raw/ALL.chr22.phase3_v5.shapeit2_mvncall_integrated.noSingleton.genotypes.vcf.gz"
@time filter_and_mask(chr22, 500) # 500 imputation samples, remaining 2004 used as reference

computing tgt/ref/record index
generating target file
generating reference file
generating masked file


Creating ./tgt_masked_unphased_chr22.vcf.gz...100%|█████| Time: 0:01:56


2101.643985 seconds (24.78 G allocations: 2.084 TiB, 9.99% gc time)


# Run MendelImpute (fast)

In [2]:
# fast method
cd("/Users/biona001/.julia/dev/MendelImpute/data/1000_genome_phase3_v5/filtered")
function run()
    X_complete = convert_gt(Float32, "tgt_chr22.vcf.gz")
    n, p = size(X_complete)
    for width in 1000
        println("running fast method, width = $width")
        tgtfile = "./tgt_masked_unphased_chr22.vcf.gz"
        reffile = "./ref_chr22.vcf.gz"
        outfile = "./mendel_imputed_fast_$(width).vcf.gz"
        @time phase(tgtfile, reffile, outfile = outfile, width = width, fast_method=true)
        X_mendel = convert_gt(Float32, outfile)
        println("error = $(sum(X_mendel .!= X_complete) / n / p)")
        X_mendel = nothing
        GC.gc()
        println("")
    end
end
run()

running width = 1000


Importing genotype and haplotype files...100%|██████████| Time: 0:03:59
Computing optimal haplotype pairs...100%|███████████████| Time: 0:28:42
Intersecting haplotypes...100%|█████████████████████████| Time: 0:00:01
Merging breakpoints...100%|█████████████████████████████| Time: 0:52:04
Writing to file...100%|█████████████████████████████████| Time: 0:01:51


5247.320492 seconds (4.92 G allocations: 460.318 GiB, 0.69% gc time)
error = 5.980410550455159e-5



# Run MendelImpute (dynamic programming)

In [2]:
Threads.nthreads()

4

In [3]:
# dynamic programming 
cd("/Users/biona001/.julia/dev/MendelImpute/data/1000_genome_phase3_v5/filtered")
function run_dp()
    X_complete = convert_gt(Float32, "tgt_chr22.vcf.gz")
    n, p = size(X_complete)
    for width in 1000
        println("running dynamic programming, width = $width")
        tgtfile = "./tgt_masked_unphased_chr22.vcf.gz"
        reffile = "./ref_chr22.vcf.gz"
        outfile = "./mendel_imputed_fast_$(width).vcf.gz"
        @time phase(tgtfile, reffile, outfile = outfile, width = width, fast_method=false)
        X_mendel = convert_gt(Float32, outfile)
        println("error = $(sum(X_mendel .!= X_complete) / n / p)")
        X_mendel = nothing
        GC.gc()
        println("")
    end
end
run_dp()

running width = 1000


Importing genotype and haplotype files...100%|██████████| Time: 0:04:14
Computing optimal haplotype pairs...100%|███████████████| Time: 0:29:11
Imputing samples...100%|████████████████████████████████| Time: 0:30:50
Writing to file...100%|█████████████████████████████████| Time: 0:01:48


4012.073367 seconds (4.91 G allocations: 461.667 GiB, 0.94% gc time)
error = 5.823806592561467e-5



# Run Beagle 5.0

In [13]:
# beagle 5
cd("/Users/biona001/.julia/dev/MendelImpute/data/1000_genome_phase3_v5/filtered")
function run_beagle()
    run(`java -Xmx15g -jar beagle.28Sep18.793.jar gt=tgt_masked_unphased_chr22.vcf.gz ref=ref_chr22.vcf.gz out=beagle_imputed, nthreads=4`)
end
run_beagle()

beagle.28Sep18.793.jar (version 5.0)
Copyright (C) 2014-2018 Brian L. Browning
Enter "java -jar beagle.28Sep18.793.jar" to list command line argument
Start time: 12:56 AM PDT on 06 Apr 2020

Command line: java -Xmx13653m -jar beagle.28Sep18.793.jar
  gt=tgt_masked_unphased_chr22.vcf.gz
  ref=ref_chr22.vcf.gz
  out=beagle_imputed,
  nthreads=4

No genetic map is specified: using 1 cM = 1 Mb

Reference samples:       2,004
Study samples:             500

Window 1 (22:16050115-51244237)
Reference markers:     644,939
Study markers:         644,939

Burnin  iteration 1:           3 minutes 25 seconds
Burnin  iteration 2:           3 minutes 59 seconds
Burnin  iteration 3:           5 minutes 13 seconds
Burnin  iteration 4:           6 minutes 47 seconds
Burnin  iteration 5:           8 minutes 20 seconds
Burnin  iteration 6:           8 minutes 55 seconds

Phasing iteration 1:           8 minutes 47 seconds
Phasing iteration 2:           8 minutes 37 seconds
Phasing iteration 3:           

Process(`java -Xmx15g -jar beagle.28Sep18.793.jar gt=tgt_masked_unphased_chr22.vcf.gz ref=ref_chr22.vcf.gz out=beagle_imputed, nthreads=4`, ProcessExited(0))

In [14]:
X_complete = convert_gt(Float32, "tgt_chr22.vcf.gz")
X_beagle = convert_gt(Float32, "beagle_imputed,.vcf.gz")
n, p = size(X_complete)
println("error = $(sum(X_beagle .!= X_complete) / n / p)")

error = 5.513389638399911e-5


## Scripts for submitting jobs on Hoffman

In [ ]:
# filtering
cd("/u/home/b/biona001/haplotype_comparisons/data") 
open("filter.sh", "w") do io
    println(io, "#!/bin/bash")
    println(io, "#\$ -cwd")
    println(io, "# error = Merged with joblog")
    println(io, "#\$ -o joblog.\$JOB_ID")
    println(io, "#\$ -j y")
    println(io, "#\$ -l exclusive,h_rt=24:00:00,h_data=20G")
    println(io, "# Email address to notify")
    println(io, "#\$ -M \$USER@mail")
    println(io, "# Notify when")
    println(io, "#\$ -m a")
    println(io)
    println(io, "echo \"Job \$JOB_ID started on:   \" `hostname -s`")
    println(io, "echo \"Job \$JOB_ID started on:   \" `date `")
    println(io)
    println(io, "# load the job environment:")
    println(io, ". /u/local/Modules/default/init/modules.sh")
    println(io, "module load julia/1.2.0")
    println(io, "module load java/1.8.0_111")
    println(io)
    println(io, "echo \'julia ./filter_and_mask.jl\' ")
    println(io, "julia /u/home/b/biona001/haplotype_comparisons/data/filter_and_mask.jl")
    println(io)
    println(io, "echo \"Job \$JOB_ID ended on:   \" `hostname -s`")
    println(io, "echo \"Job \$JOB_ID ended on:   \" `date `")
    println(io)
end
# submit job
run(`qsub filter.sh`)
rm("filter.sh", force=true)
    

In [ ]:
# convert to m3vcf file using minimac3
cd("/u/home/b/biona001/haplotype_comparisons/data") 
open("minimac3.sh", "w") do io
    println(io, "#!/bin/bash")
    println(io, "#\$ -cwd")
    println(io, "# error = Merged with joblog")
    println(io, "#\$ -o joblog.\$JOB_ID")
    println(io, "#\$ -j y")
    println(io, "#\$ -l exclusive,h_rt=24:00:00,h_data=20G")
    println(io, "# Email address to notify")
    println(io, "#\$ -M \$USER@mail")
    println(io, "# Notify when")
    println(io, "#\$ -m a")
    println(io)
    println(io, "echo \"Job \$JOB_ID started on:   \" `hostname -s`")
    println(io, "echo \"Job \$JOB_ID started on:   \" `date `")
    println(io)
    println(io, "# load the job environment:")
    println(io, ". /u/local/Modules/default/init/modules.sh")
    println(io, "module load julia/1.2.0")
    println(io, "module load R/3.5.1")
    println(io, "module load java/1.8.0_111")
    println(io)
    println(io, "echo \'converting chr22, chr21, chr20 from vcf.gz to m3vcf.gz\' ")
    println(io, "/u/home/b/biona001/haplotype_comparisons/Minimac3/bin/Minimac3 --refHaps ref_chr22.vcf.gz --processReference --prefix ref_chr22")
    println(io, "/u/home/b/biona001/haplotype_comparisons/Minimac3/bin/Minimac3 --refHaps ref_chr21.vcf.gz --processReference --prefix ref_chr21")
    println(io, "/u/home/b/biona001/haplotype_comparisons/Minimac3/bin/Minimac3 --refHaps ref_chr20.vcf.gz --processReference --prefix ref_chr20")
    println(io)
    println(io, "echo \"Job \$JOB_ID ended on:   \" `hostname -s`")
    println(io, "echo \"Job \$JOB_ID ended on:   \" `date `")
    println(io)
end
# submit job
run(`qsub minimac3.sh`)
rm("minimac3.sh", force=true)

In [ ]:
# run mendel dp and fast method
cd("/u/home/b/biona001/haplotype_comparisons/data") 
open("mendel.sh", "w") do io
    println(io, "#!/bin/bash")
    println(io, "#\$ -cwd")
    println(io, "# error = Merged with joblog")
    println(io, "#\$ -o joblog.\$JOB_ID")
    println(io, "#\$ -j y")
    println(io, "#\$ -l arch=intel-X5650,exclusive,h_rt=24:00:00,h_data=20G")
    println(io, "# Email address to notify")
    println(io, "#\$ -M \$USER@mail")
    println(io, "# Notify when")
    println(io, "#\$ -m a")
    println(io)
    println(io, "echo \"Job \$JOB_ID started on:   \" `hostname -s`")
    println(io, "echo \"Job \$JOB_ID started on:   \" `date `")
    println(io)
    println(io, "# load the job environment:")
    println(io, ". /u/local/Modules/default/init/modules.sh")
    println(io, "module load julia/1.2.0")
    println(io, "module load java/1.8.0_111")
    println(io)
    println(io, "echo \'run mendel (dp and fast method) on chr 20, 21. width = 400, 800, 1200, 1600, 12 threads\' ")
    println(io, "export JULIA_NUM_THREADS=12")
    println(io, "julia mendel.jl")
    println(io)
    println(io, "echo \"Job \$JOB_ID ended on:   \" `hostname -s`")
    println(io, "echo \"Job \$JOB_ID ended on:   \" `date `")
    println(io)
end
# submit job
run(`qsub mendel.sh`)
rm("mendel.sh", force=true)

In [ ]:
# run beagle 5
cd("/u/home/b/biona001/haplotype_comparisons/data") 
open("beagle.sh", "w") do io
    println(io, "#!/bin/bash")
    println(io, "#\$ -cwd")
    println(io, "# error = Merged with joblog")
    println(io, "#\$ -o joblog.\$JOB_ID")
    println(io, "#\$ -j y")
    println(io, "#\$ -l arch=intel-X5650,exclusive,h_rt=24:00:00,h_data=20G")
    println(io, "# Email address to notify")
    println(io, "#\$ -M \$USER@mail")
    println(io, "# Notify when")
    println(io, "#\$ -m a")
    println(io)
    println(io, "echo \"Job \$JOB_ID started on:   \" `hostname -s`")
    println(io, "echo \"Job \$JOB_ID started on:   \" `date `")
    println(io)
    println(io, "# load the job environment:")
    println(io, ". /u/local/Modules/default/init/modules.sh")
    println(io, "module load julia/1.2.0")
    println(io, "module load java/1.8.0_111")
    println(io)
    println(io, "echo \'run beagle 5.0 on chr 20, 21 with 12 threads\' ")
    println(io, "java -Xmx$42g -jar beagle5.0.jar gt=tgt_masked_unphased_chr20.vcf.gz ref=ref_chr20.vcf.gz out=beagle_imputed_chr20 nthreads=12")
    println(io, "java -Xmx$42g -jar beagle5.0.jar gt=tgt_masked_unphased_chr21.vcf.gz ref=ref_chr21.vcf.gz out=beagle_imputed_chr21 nthreads=12")
    println(io)
    println(io, "echo \"Job \$JOB_ID ended on:   \" `hostname -s`")
    println(io, "echo \"Job \$JOB_ID ended on:   \" `date `")
    println(io)
end
# submit job
run(`qsub beagle.sh`)
rm("beagle.sh", force=true)

In [ ]:
# prephase using beagle 5 (need to check if output indeed retained missing data)
cd("/u/home/b/biona001/haplotype_comparisons/data") 
open("beagle.sh", "w") do io
    println(io, "#!/bin/bash")
    println(io, "#\$ -cwd")
    println(io, "# error = Merged with joblog")
    println(io, "#\$ -o joblog.\$JOB_ID")
    println(io, "#\$ -j y")
    println(io, "#\$ -l arch=intel-X5650,exclusive,h_rt=24:00:00,h_data=20G")
    println(io, "# Email address to notify")
    println(io, "#\$ -M \$USER@mail")
    println(io, "# Notify when")
    println(io, "#\$ -m a")
    println(io)
    println(io, "echo \"Job \$JOB_ID started on:   \" `hostname -s`")
    println(io, "echo \"Job \$JOB_ID started on:   \" `date `")
    println(io)
    println(io, "# load the job environment:")
    println(io, ". /u/local/Modules/default/init/modules.sh")
    println(io, "module load julia/1.2.0")
    println(io, "module load java/1.8.0_111")
    println(io)
    println(io, "echo \'prephase using beagle 5.0 on chr 20, 21, 22 with 12 threads\' ")
    println(io, "java -Xmx$42g -jar beagle5.0.jar gt=tgt_masked_unphased_chr20.vcf.gz ref=ref_chr20.vcf.gz out=tgt_masked_prephased_chr20 niteration=0 impute=false nthreads=12")
    println(io, "java -Xmx$42g -jar beagle5.0.jar gt=tgt_masked_unphased_chr21.vcf.gz ref=ref_chr21.vcf.gz out=tgt_masked_prephased_chr21 niteration=0 impute=false nthreads=12")
    println(io, "java -Xmx$42g -jar beagle5.0.jar gt=tgt_masked_unphased_chr22.vcf.gz ref=ref_chr21.vcf.gz out=tgt_masked_prephased_chr22 niteration=0 impute=false nthreads=12")
    println(io)
    println(io, "echo \"Job \$JOB_ID ended on:   \" `hostname -s`")
    println(io, "echo \"Job \$JOB_ID ended on:   \" `date `")
    println(io)
end
# submit job
run(`qsub beagle.sh`)
rm("beagle.sh", force=true)

In [ ]:
# run minimac 4
cd("/u/home/b/biona001/haplotype_comparisons/data") 
open("beagle.sh", "w") do io
    println(io, "#!/bin/bash")
    println(io, "#\$ -cwd")
    println(io, "# error = Merged with joblog")
    println(io, "#\$ -o joblog.\$JOB_ID")
    println(io, "#\$ -j y")
    println(io, "#\$ -l arch=intel-X5650,exclusive,h_rt=24:00:00,h_data=20G")
    println(io, "# Email address to notify")
    println(io, "#\$ -M \$USER@mail")
    println(io, "# Notify when")
    println(io, "#\$ -m a")
    println(io)
    println(io, "echo \"Job \$JOB_ID started on:   \" `hostname -s`")
    println(io, "echo \"Job \$JOB_ID started on:   \" `date `")
    println(io)
    println(io, "# load the job environment:")
    println(io, ". /u/local/Modules/default/init/modules.sh")
    println(io, "module load julia/1.2.0")
    println(io, "module load java/1.8.0_111")
    println(io)
    println(io, "echo \'run minimac4 on chr 20, 21, 22 with 12 threads\' ")
    println(io, "minimac4 --refHaps ref_chr20.vcf.m3vcf.gz --haps tgt_masked_prephased_chr20.vcf.gz --prefix minimac_imputed_chr20 --format GT --cpus 12")
    println(io, "minimac4 --refHaps ref_chr21.vcf.m3vcf.gz --haps tgt_masked_prephased_chr21.vcf.gz --prefix minimac_imputed_chr21 --format GT --cpus 12")
    println(io, "minimac4 --refHaps ref_chr22.vcf.m3vcf.gz --haps tgt_masked_prephased_chr22.vcf.gz --prefix minimac_imputed_chr22 --format GT --cpus 12")
    println(io)
    println(io, "echo \"Job \$JOB_ID ended on:   \" `hostname -s`")
    println(io, "echo \"Job \$JOB_ID ended on:   \" `date `")
    println(io)
end
# submit job
run(`qsub beagle.sh`)
rm("beagle.sh", force=true)